In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

/home/kdd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kdd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kdd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kdd/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

1.14.0
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.18.1
pandas 0.24.1
sklearn 0.21.2
tensorflow 1.14.0
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
print(x_train_all.shape, y_train_all.shape)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1, 28*28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1, 28*28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1, 28*28)

y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)
y_valid = np.asarray(y_valid, dtype=np.int64)

In [5]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [6]:
batch_size = 20
epochs = 10

images_placeholder = tf.placeholder(tf.float32, [None, 28*28])
labels_placeholder = tf.placeholder(tf.int64, (None, ))

dataset = make_dataset(images_placeholder, labels_placeholder,
                       epochs=epochs, batch_size=batch_size)
dataset_iter = dataset.make_initializable_iterator()
x, y = dataset_iter.get_next()

# with tf.Session() as sess:
#     sess.run(dataset_iter.initializer,
#              feed_dict={
#                  images_placeholder: x_train_scaled,
#                  labels_placeholder: y_train
#              })
#     x_val, y_val = sess.run([x, y])
#     print(x_val)
#     print(y_val)
#     sess.run(dataset_iter.initializer,
#          feed_dict={
#              images_placeholder: x_valid_scaled,
#              labels_placeholder: y_valid
#          })
#     x_val, y_val = sess.run([x, y])
#     print(x_val)
#     print(y_val)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [7]:
hidden_units = [100, 100]
class_num = 10

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(inputs=input_for_next_layer,
                                           units=hidden_unit,
                                           activation=tf.nn.relu)
logits = tf.layers.dense(inputs=input_for_next_layer, units=class_num)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)
prediction = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y), tf.float32))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
print(logits)

Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [9]:
init_op = tf.global_variables_initializer()

trian_steps_per_epoch = x_train_scaled.shape[0] // batch_size
valid_steps = x_valid_scaled.shape[0] // batch_size
test_steps = x_test_scaled.shape[0] // batch_size


In [12]:
def evaluate(sess, x, y, accuracy, images, labels, batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        feature_batch = images[step * batch_size : (step+1) * batch_size]
        label_batch = labels[step * batch_size : (step+1) * batch_size]
        accuracy_val = sess.run(accuracy, feed_dict={
            x : feature_batch,
            y : label_batch
        })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(epochs):
        for step in range(trian_steps_per_epoch):
            sess.run(dataset_iter.initializer, feed_dict={
                images_placeholder: x_train_scaled,
                labels_placeholder: y_train
            })
            loss_val, accuracy_val, _ = sess.run([loss, accuracy, train_op])
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f' %(
                epoch, step, loss_val, accuracy_val), end='')
        sess.run(dataset_iter.initializer, feed_dict={
            images_placeholder: x_valid_scaled,
            labels_placeholder: y_valid
            })
        valid_accuracy = evaluate(sess, x, y, accuracy, x_valid_scaled, y_valid, batch_size)
        print('\t[Valid] acc: %2.2f' % valid_accuracy)


[Train] epoch: 0, step: 1305, loss: 0.37001, accuracy: 0.85

KeyboardInterrupt: 